<a name="Exercise"></a>

### /!\ J'ai eu quelques soucis en terme de stockage Github donc j'ai dû mettre une partie du script sous forme de commentaire pour limiter les outputs volumineux /!\

<a name="Exercise"></a>

### Exercises

#### CRUD operations

Create a new db name Todo and a new collection named "CRUD_exercise" and do the following:

In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['Todo']
collection = db['CRUD_exercise']
print("Database and Collection Created: ", db.name, collection.name)

Database and Collection Created:  Todo CRUD_exercise


**TODO 1**: Take the dict created in the TODO 4 in chapter I and save it in the collection "CRUD_exercise".

In [1]:
import sqlite3

conn = sqlite3.connect('/Users/valen/Desktop/SAS/BC.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM tomatch3")
rows = cursor.fetchall()

data = [{'id': row[0], 'rdm_float': row[1]} for row in rows]

conn.close()

In [2]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['Todo']
collection = db['CRUD_exercise']

collection.insert_many(data)

print(collection.name)

Data inserted into MongoDB collection: CRUD_exercise


**TODO 2**: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

In [3]:
#Insert 3 documents with key = x and values = 1
docs = [{'x': 1}, {'x': 1}, {'x': 1}]
insert_result = collection.insert_many(docs)
print(f"Inserted document IDs: {insert_result.inserted_ids}")

Inserted document IDs: [ObjectId('6630ee93e01c5a30842e1678'), ObjectId('6630ee93e01c5a30842e1679'), ObjectId('6630ee93e01c5a30842e167a')]


In [4]:
#delete one of them
delete_result = collection.delete_one({'x': 1})
print(f"Deleted count: {delete_result.deleted_count}")

Deleted count: 1


MongoDB ne va pas prendre en compte le plus récent ou plus ancien des documents mais supprimer le premier qu'il rencontre ayant la caractéristique demandé, dans notre cas, nous avons trois documents "{'x': 1}", donc il va en supprimer un au hasard s'il n'y a pas plus de spécification

In [5]:
#increment the value of x to 4
update_result = collection.update_one({'x': 1}, {'$set': {'x': 4}})
print(f"Updated count: {update_result.modified_count}")

Updated count: 1


**TODO 3**: Insert the dict created in the TODO 6 Chapter I in the example collection.

In [2]:
import sqlite3

conn = sqlite3.connect('/Users/valen/Desktop/SAS/BC.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM NN")
rows = cursor.fetchall()

dataNN = [{'id': row[0], 'rdm_float': row[1]} for row in rows]

conn.close()

In [4]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['Todo']
collection = db['CRUD_exercise']

collection.insert_many(dataNN)

print(collection.name)

CRUD_exercise


**TODO 4**: Get documents where authors key exist in the collection "CRUD_exercise".

In [18]:
documents = collection.find({'authors': {'$exists': True}})

for doc in documents:
    print(doc)


**TODO 5**: Change the documents where x = 4 to x = 1.

In [21]:
collection.update_many({'x': 4}, {'$set': {'x': 1}})


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)


**TODO 6**: Find documents where author is not_mike and set author as real_mike.

In [22]:
collection.update_many({'author': 'not_mike'}, {'$set': {'author': 'real_mike'}})

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)


**TODO 7**: Delete documents where author is real_mike.

In [23]:
collection.delete_many({'author': 'real_mike'})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

#### Managing DB

**TODO 8**: create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

In [6]:
import time
import numpy as np
start_time_no_index = time.time()
doc = [{'id': i, 'values': np.random.rand(10).tolist()} for i in range(0, 1000000, 2)]

insert_result = collection.insert_many(doc)
end_time_no_index = time.time()
print(f"Inserted {len(insert_result.inserted_ids)} doc.")

Inserted 500000 doc.


In [7]:
import pymongo
start_time_with_index = time.time()
collection.create_index([('id', pymongo.ASCENDING)])
end_time_with_index = time.time()

In [10]:
print(f"Sans index: {end_time_no_index - start_time_no_index} secondes.")
print(f"Avec index: {end_time_with_index - start_time_with_index} secondes.")
print(f"...soit un ratio de: {(end_time_no_index - start_time_no_index)/(end_time_with_index - start_time_with_index)}")

Sans index: 13.77726674079895 secondes.
Avec index: 0.003922224044799805 secondes.
...soit un ratio de: 3512.615950398152


Oui, nous remarquons que la requête sans index mets beaucoup plus de temps que la requête avec index

**TODO 9**: create a random collection in a random db and put the new collection in the tutorial DB

In [41]:
import pymongo
import numpy as np
from pymongo import MongoClient
import string
import random

def random_string(length=10):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

client = MongoClient('localhost', 27017)

random_db_name = random_string()
random_col_name = random_string()

random_db = client[random_db_name]
random_collection = random_db[random_col_name]

documents = [{'id': i, 'values': np.random.rand(10).tolist()} for i in range(10)]
random_collection.insert_many(documents)
print(f"Data inserted into {random_db_name}.{random_col_name}")

tutorial_db = client['tutorial']
tutorial_collection = tutorial_db[random_col_name]

data_to_transfer = list(random_collection.find({}))
tutorial_collection.insert_many(data_to_transfer)
print(f"Data copied to tutorial.{random_col_name}")

client.drop_database(random_db_name)
print(f"Database {random_db_name} deleted")

print("Collections in 'tutorial' database:", tutorial_db.list_collection_names())

Data inserted into zwroewvwme.erqrbfzety
Data copied to tutorial.erqrbfzety
Database zwroewvwme deleted
Collections in 'tutorial' database: ['image', 'benchmark_2', 'example', 'benchmark', 'arxiv_api', 'erqrbfzety', 'example_to_dump']


**TODO 10**: What is the difference between an inner join and an outer join ? Is the query seen during course an inner or outer join ? Play with the query to show all the joins.

#### Real world problems

**TODO 11**:  Use the oaipmh and api code get papers after January 2020 and for "cs,math,econ" categories. Insert them in MongoDB. Import only the first 200. How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "cs".

**TODO 12**: Do the same as TODO 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.


**TODO 13**: Download a random image and store it in a collection.

In [22]:
import requests

img = "https://i.redd.it/5hede1edzl561.jpg"

response = requests.get(img)

if response.status_code == 200:
    with open("LearnPythonMeme.jpg", "wb") as file:
        file.write(response.content)

In [28]:
import base64
from pymongo import MongoClient

#MongoDB étant une base orientée document, nous aurons besoin de convertir la donnée binaire 
#en chaine de caractère ASCII grace à l'encodage en base64

fp = "/Users/valen/Documents/GitHub/ExercicesNoSQL/EXERCICES/LearnPythonMeme.jpg"
with open(fp, "rb") as LPM:
    encoded_string = base64.b64encode(LPM.read())

client = MongoClient('mongodb://localhost:27017/')
db = client["mydatabase"]
collection = db["images"]

image_document = {
    "image_name": fp.split("/")[-1],
    "image_data": encoded_string
}

result = collection.insert_one(image_document)
result.inserted_id

ObjectId('66312ad1ea969c4b1a2070a8')


**TODO 14**: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)


In [54]:
import pandas as pd
from pymongo import MongoClient

data = {'Video game': ['Elden Ring', 'Metal Gear Solid: Sons of Liberty', 'Devil May Cry'],
        'Year': [2022, 2001, 2001],
        'Studio': ['FromSoftware ', 'Konami', 'Capcom']}
df = pd.DataFrame(data)

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['VideoGames']


records = df.to_dict('records')
result = collection.insert_many(records)

print(result.inserted_ids)

[ObjectId('6631fa0aea969c4b1a7dbb3b'), ObjectId('6631fa0aea969c4b1a7dbb3c'), ObjectId('6631fa0aea969c4b1a7dbb3d')]


In [70]:
df

,Video game,Year,Studio
0,Elden Ring,2022,FromSoftware
1,Metal Gear Solid: Sons of Liberty,2001,Konami
2,Devil May Cry,2001,Capcom


**TODO 15**: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.


**TODO 16**: Download a [sound sample](https://freesound.org/browse/). Try to store it in MongoDB 

In [39]:
import wave
import base64

#Pour les mêmes raisons évoquée pour le stockage de l'image, nous devrons procéder 
#à un encodage en base64 pour stocker le fichier ".wav" dans MongoDB
wav_fp = '/Users/valen/Desktop/data/Chap3/734019__klankbeeld__storm-wind-in-trees-528-pm-160221_0857.wav'

with wave.open(wav_fp, 'rb') as wav_file:
    frames = wav_file.readframes(wav_file.getnframes())

encoded_audio = base64.b64encode(frames).decode('utf-8')

In [42]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['audio_files']

audio_document = {
    "file_name": wav_fp.split("/")[-1],
    "audio_data": encoded_audio,
    "num_channels": wav_file.getnchannels(),
    "sample_width": wav_file.getsampwidth(),
    "frame_rate": wav_file.getframerate(),
    "num_frames": wav_file.getnframes()
}

collection.insert_one(audio_document)



InsertOneResult(ObjectId('663137b4ea969c4b1a2070b1'), acknowledged=True)

**TODO 17**: Create a collection with 30M observation with a single key : "year" which is a random value between 2000-2020. Get documents with year = 2000. Does using an index helps ? 

In [49]:
import pymongo
import random
from pymongo import MongoClient
import time

client = MongoClient('mongodb://localhost:27017/')
db = client['large_data']
collection = db['years']

batch_size = 100000
for _ in range(300):  # Insertion de 30 millions de documents
    batch = [{"year": random.randint(2000, 2020)} for _ in range(batch_size)]
    collection.insert_many(batch)

if "year_1" not in collection.index_information(): 
    collection.create_index([("year", pymongo.ASCENDING)])

# Mesure de temps pour la query sans index
start_time_no_index = time.time()
count_no_index = collection.count_documents({"year": 2000})
time_no_index = time.time() - start_time_no_index
print("Temps sans index:", time_no_index, "secondes, Nombre:", count_no_index)

# Mesure de temps pour la query avec index (year)
start_time_with_index = time.time()
count_with_index = collection.count_documents({"year": 2000})
time_with_index = time.time() - start_time_with_index
print("Temps avec index:", time_with_index, "secondes, Nombre:", count_with_index)

client.close()


Temps sans index: 4.274502515792847 secondes, Nombre: 4287432
Temps avec index: 4.032424688339233 secondes, Nombre: 4287432


### Homework 1

pubmed_cleaned.zip is a file containing a metadatas sample from pubmed articles. Your goal is to **convert the json to a mongo DB** and answer the following questions **USING** mongodb querys:

In [15]:
import json
from bson import ObjectId
from pymongo import MongoClient

#Après essaies, j'ai remarqué que le fichier contenait un caractère spécial qui n'est pas
#accepté par mongoDB dans le fichier mongoDB, il faut donc rempalcer "$oid" par un autre
#nom ID

def replace_oid(data):
    """ Recursively replace '$oid' with ObjectId """
    if isinstance(data, dict):
        if '$oid' in data:
            return ObjectId(data['$oid'])
        return {k: replace_oid(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [replace_oid(v) for v in data]
    else:
        return data

json_file_path = "/Users/valen/Desktop/data/Chap3/pubmed_cleaned/pubmed_cleaned.json"

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']  
collection = db['pubmed_data'] 

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)  

    processed_data = replace_oid(data)

    if isinstance(processed_data, list):
        collection.insert_many(processed_data)
    else:
        collection.insert_one(processed_data)



1) Create an index, explain your choice of key.

In [16]:
#Nous indexerons l'id "pmid"
collection.create_index([("pmid", 1)])

'pmid_1'

2) Delete every paper that was published prior 2019

In [18]:
from datetime import datetime

def convert_string_to_datetime(date_string):
    year, month, day, hour, minute = map(int, date_string.split(","))
    return datetime(year, month, day, hour, minute)

date_reference = datetime(2019, 1, 1)

result = collection.delete_many({"date": {"$lt": date_reference}})

print(f"{result.deleted_count} articles ont été supprimés.")

0 articles ont été supprimés.


3) How many paper have a single author ? Two authors ?


In [19]:
single_author_count = collection.aggregate([
    {"$match": {"authors": {"$exists": True}}}, 
    {"$project": {"num_authors": {"$size": {"$split": ["$authors", ","]}}}},
    {"$match": {"num_authors": 1}},
    {"$count": "count"}
])

two_authors_count = collection.aggregate([
    {"$match": {"authors": {"$exists": True}}}, 
    {"$project": {"num_authors": {"$size": {"$split": ["$authors", ","]}}}}, 
    {"$match": {"num_authors": 2}},
    {"$count": "count"} 
])

single_author_count = list(single_author_count)[0]["count"] if single_author_count.alive else 0
two_authors_count = list(two_authors_count)[0]["count"] if two_authors_count.alive else 0

print(f"Nombre de documents avec un seul auteur : {single_author_count}")
print(f"Nombre de documents avec deux auteurs : {two_authors_count}")

Nombre de documents avec un seul auteur : 580
Nombre de documents avec deux auteurs : 142


4) What's the last paper inserted in the db ?


In [20]:
last_paper = collection.find_one({}, sort=[('_id', -1)])

print(last_paper)

{'_id': ObjectId('60c753bac4b7559e040fe2b3'), 'pmid': 31226374, 'title': 'Identification of a novel UDP-glycosyltransferase gene from Rhodiola rosea and its expression during biotransformation of upstream precursors in callus culture.', 'ISSN': '1879-0003', 'abstract': ' "Roseroot (Rhodiola rosea L.) is a medicinal plant with adaptogenic properties and several pharmaceutically important metabolites. In this study, a full length cDNA encoding a UDPG gene of roseroot was identified, cloned and characterized. Its ORF (1425bp) was transferred into E. coli, where the expression of the recombinant enzyme was confirmed. To monitor the enzyme activity, 3 precursors (tyramine, 4-hydroxyphenylpyruvate & tyrosol) of salidroside biosynthesis pathway were added to roseroot callus cultures and samples were harvested after 1, 6, 12, 24, 48 & 96h. Along with the controls (without precursor feeding), each sample was subjected to HPLC and qRT-PCR for phytochemical and relative UDP-glycosyltransferase ge

5) Find articles with null meshwords.

In [3]:
articles_with_null_meshwords = collection.find({"meshwords": None})

#for article in articles_with_null_meshwords:
    #print(article)

6) Choose a keyword you are interested in (machine learning, computer vision,...). Find the number of articles with the choosen keyword in their meshwords, abstract or title.


In [31]:
#Nous allons choisir "Green technology"

import re

#Nous nous préoccupons du cas ou il s'agit de plusieurs technologies, en utilisant une boucle for
keywords = ["green technology", "green technologies"]

regex_patterns = [re.compile(keyword, re.IGNORECASE) for keyword in keywords]

articles_count = collection.count_documents({
    "$or": [
        {"meshwords": {"$in": [pattern for pattern in regex_patterns]}},
        {"abstract": {"$in": [pattern for pattern in regex_patterns]}},
        {"title": {"$in": [pattern for pattern in regex_patterns]}}
    ]
})

articles_count


18

7) What's the number of articles that have atleast one affiliation AND meshwords.


In [34]:
articles_count = collection.count_documents({
    "authors": {"$regex": "affil", "$options": "i"},
    "meshwords": {"$exists": True, "$ne": None}
})

articles_count

74932

8) How many articles have a publishing date after 2020 ?


9) Find articles where there's atleast one affiliation from a choosen country (you decide which one).


In [35]:
chosen_country = "Netherlands"

articles_count = collection.count_documents({
    "authors": {"$regex": chosen_country, "$options": "i"}})

articles_count

2828

10) Check for any duplicates. (hint: look at the doi or the pmid)


In [39]:
duplicates_by_doi = list(collection.aggregate([
    {"$match": {"doi": {"$ne": None}}},
    {"$group": {"_id": "$doi", "count": {"$sum": 1}}},
    {"$match": {"count": {"$gt": 1}}}
]))

print("Doublons basés sur le champ DOI :")
for duplicate in duplicates_by_doi:
    print(f"DOI: {duplicate['_id']}, Nbr :{duplicate['count']}")

Doublons basés sur le champ DOI :
DOI: 10.1093/cid/ciz518, Nbr :2
DOI:  }, authors { names std { { name ml , Nbr :73
DOI: , Nbr :5
DOI: 10.1093/jas/skz192, Nbr :2
DOI: , affil str , Nbr :40
DOI:  } } }, from journal { title { iso-jta , Nbr :10
DOI:  }, { name ml , Nbr :55
DOI: 10.1093/jnci/djz062, Nbr :2
DOI: 10.23876/j.krcp.19.006, Nbr :2


11) Remove every articles where the abstract starts with an "R".


In [ ]:
#collection.delete_many({"abstract": {"$regex": "^R", "$options": "i"}})

12) Return the list of papers (pmid) where there's atleast one affiliation per author

In [43]:
papers_with_one_affil_per_author_count = 0

papers_with_one_affil_per_author = []

articles = collection.find({"$expr": {"$eq": [{"$size": {"$split": ["$authors", "affil"]}}, "$team_size"]}})

for article in articles:
    papers_with_one_affil_per_author_count += 1
    papers_with_one_affil_per_author.append(article["pmid"])

print(papers_with_one_affil_per_author_count)
print("Liste des PMIDs des articles répondant aux critères:")
print(papers_with_one_affil_per_author)

272
Liste des PMIDs des articles répondant aux critères:
[30647233, 30647235, 30647237, 30647238, 30647242, 30647243, 30647250, 30647261, 30647264, 30647267, 30647269, 30647271, 30647273, 30647280, 30647281, 30647287, 30647288, 30647291, 30647298, 30647299, 30647303, 30647305, 30647308, 30647309, 30647313, 30647314, 30647315, 30647318, 30647319, 30697754, 30697762, 30706834, 30706835, 30710987, 30710991, 30715029, 30715030, 30715036, 30717343, 30737981, 30759836, 30772820, 30790789, 30790793, 30790797, 30790801, 30790802, 30790803, 30790807, 30790809, 30790810, 30790812, 30790816, 30790817, 30790819, 30790822, 30790826, 30790831, 30790839, 30790845, 30790848, 30790852, 30790855, 30790861, 30790863, 30790864, 30790865, 30790870, 30790873, 30790874, 30790878, 30790880, 30790881, 30790884, 30790889, 30790892, 30790895, 30790898, 30790899, 30790902, 30790908, 30790910, 30790912, 30790918, 30790919, 30790921, 30790922, 30790926, 30790928, 30790930, 30790933, 30790934, 30790936, 30790938, 30


13) Create 500 random samples of the dataset, compute a statistics that you are interested in and check how it behaves through the different samples


14) Sandbox exercise: think of a problematic and try to answer it.

### Homework 2

authors.zip is a file containing a sample of authors that wrote a paper published on pubmed. Each doc as, at most, 5 keys. "AND_ID" is the disambiguated author id. "pmid_list" is the list of ids that the author published. "more_info" is a list of dict with each dict representing info for a given paper. "oa04_affiliations" is a list of dict with each dict representing affiliation info for a given paper. "oa06_researcher_education" is a list of dict with each dict containing information on the education of the researcher.

Your goal is to **convert the json to a mongo DB** and answer the following questions **USING** mongodb querys:

In [1]:
import json
from bson import ObjectId
from pymongo import MongoClient

authors_fp = "/Users/valen/Desktop/data/Chap3/authors.json"

#Nous avons utilisé la même fonction lors de l'exercice précédent
def replace_oid(data):
    """ Recursively replace '$oid' with ObjectId """
    if isinstance(data, dict):
        if '_id' in data and '$oid' in data['_id']:
            data['_id'] = ObjectId(data['_id']['$oid'])
        for key, value in data.items():
            data[key] = replace_oid(value)
    elif isinstance(data, list):
        for i, item in enumerate(data):
            data[i] = replace_oid(item)
    return data

with open(authors_fp, 'r', encoding='utf-8') as f:
    data = json.load(f)

data = replace_oid(data)

client = MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['authors']

#collection.insert_many(data)

1) Create an index, explain your choice of key.


2) What is the average length of "pmid_list"


In [60]:
pipeline = [
    {"$match": {"pmid_list": {"$exists": True, "$type": "array"}}},
    {"$project": {"pmid_list_size": {"$size": "$pmid_list"}}},
    {"$group": {"_id": None, "average_pmid_list_size": {"$avg": "$pmid_list_size"}}}
]

list(collection.aggregate(pipeline))

[{'_id': None, 'average_pmid_list_size': 7.040832449946997}]

3) How many distinct affiliations are there ?

In [70]:
pipeline = [
    {"$unwind": "$oa04_affiliations"},
    {"$group": {"_id": "$oa04_affiliations.Affiliation"}},
    {"$group": {"_id": None, "count": {"$sum": 1}}}
]

result = list(collection.aggregate(pipeline))

if result:
    distinct_affiliations_count = result[0]["count"]
    print(f"There are {distinct_affiliations_count} distinct affiliations.")
else:
    print("No affiliations found.")

There are 318595 distinct affiliations.


4) Find authors with atleast one "COM" AffiliationType


In [ ]:
query = {
    "oa04_affiliations": {
        "$elemMatch": {
            "AffiliationType": "COM"
        }
    }
}

authors_with_com_affiliation = collection.find(query)

#for author in authors_with_com_affiliation:
#    print(author)

5) How many authors switched the AffiliationType ?


6) Find affiliation with the word "China" 


In [89]:
query = {"oa04_affiliations.Affiliation": {"$regex": "China", "$options": "i"}}

matching_documents = collection.find(query)

#for document in matching_documents:
    #affiliations = document.get("oa04_affiliations", [])
    #for affiliation in affiliations:
        #if "China" in affiliation.get("Affiliation", ""):
            #print(affiliation)

7) Get the pmids of papers published in 2019


In [2]:
query = {"more_info.0.PubYear": 2019}

matching_documents = collection.find(query)

pmids_2019 = []

for document in matching_documents:
    pmids = document.get("pmid_list", [])
    pmids_2019.extend(pmids)

#print(pmids_2019)

8) Count the number of doc with "oa06_researcher_education" OR "oa04_affiliations" key and with the "oa06_researcher_education" AND "oa04_affiliations" .


9) What's the average "BeginYear" of "oa06_researcher_education".


10) Count the distinct country of "oa06_researcher_education"


11) Does the length of pmid_list and more_info always match ?


12) Does the length of pmid_list and "oa04_affiliations" always match ?


13) Sandbox exercise: think of a problematic and try to answer it.